# Автоматическая обработка естественного языка
## Семантика, 2 модуль, 4 курс
### Екатерина Владимировна Еникеева, <br/>Александр Викторович Орлов

https://github.com/named-entity/hse-nlp/tree/master/4th_year#содержание

## 2023/11/8

# О чем этот курс
- прикладные задачи NLP на уровне текста
- **данные**: принципы разметки, датасеты
- **методы**: не очень подробно, акцент на реализации
- **оценка**: бенчмарки и метрики

# Основные темы
**Обработка ЕЯ на уровне текста:**
* выделение ключевых слов
* разрешение лексической омонимии (WSD)
* распознавание именованных сущностей (NER)
* разрешение анафоры / кореференции
* оценка тональности, извлечение мнений
* суммаризация
* машинный перевод
* NEW🔥 распознавание и синтез речи 

# Методы

* count-based
* classic ML + feature engineering
* embeddings-based + Deep Learning

# Основные активности
* лекции (с практической частью) - понедельник
* семинары - среда
* домашние задания (2)
* квизы по материалам лекций (3)
* финальный проект (1)

# Оценка
$$0,2 * квизы + 0,4 * домашки + 0,4 * проект$$

## Анализ значения — разнородные задачи
* выделение значимых единиц:
  * multiword expressions - MWE
  * named entities - NE, ...
* извлечение информации:
  * opinion mining
  * fact / information extraction


## Анализ значения — разнородные задачи
* анализ связи между лексич.единицами и их значениями:
  * коллокации
  * word sense disambiguation - WSD
* анализ дискурса:
  * анафора, кореференция
  * риторические отношения

## Использование ключевых слов
* информационный поиск
* выделение ключевых слов / сущностей новостных текстах и т.п.
* суммаризация текстов
* рубрикация, выделение тем, персонализация
* лексикографические ресурсы
* ...

## Использование ключевых слов
### Продуктовые штуки
* мониториг соцсетей / упоминаний брендов / фидбека пользователей
* продуктовая аналитика
* SEO
* ...

## Что считать ключевыми словами?
Давайте сравним:

https://forms.gle/WvRyZ8GxUgaGxB6MA


## Что можно оценить
- пересечение аннотаций

Если есть эталон:
- точность
- полнота


## Определение ключевых слов

1. Тематически значимые слова - отличающие текст от других текстов корпуса;
2. Значимые слова для стиля / жанра;
3. Типичные для предметной области (подкорпуса);
4. Внутри текста, независимо от остальных текстов.

## Подходы
(традиционно)
* supervised
* unsupervised

## Действующие лица
* текст / документ
* корпус / коллекция
* слово / термин / терм (может быть multiword)
* из инфопоиска: пользовательский запрос, релевантность


### Пайплайн

![KW_Pipeline](kw-1.png)

## Классификация методов
![KW-Approaches](kw-2.jpeg)

# Статистические методы

## Ключевые слова в IR
* цель: сравнить запрос с документом
* самый распространённый способ: $tf*idf$
  * **??** тексты небольшого объёма
  * ?? новостные тексты - много общеупотребительных слов
  * ~ тематический вес слова (относительно других текстов)

## Okapi BM25

$$score(D, Q) = \sum_{i=1}^{n}{IDF(q_i)*\frac{f(q_i, D)*(k_1 + 1)}{f(q_i, D)+k_1*(1-b+b*\frac{|D|}{avgLen})}}$$
где

$q_i$ - $i$-е слово в запросе

$f(q_i, D)$ - частота $q_i$ в документе $D$

$k_1$, $b$ - коэффициенты

$|D|$ - длина документа

$avgLen$ - средняя длина документа в корпусе


## Контрастивные методы
- сравниваем распределения частот: 
  * в целевом (target) корпусе -- O (observed) frequencies 
  * в контрастном (reference) -- E (expected) frequencies

- можем использовать статистические критерии проверки гипотез:
  - $\chi^2$
  - $log-likelihood$
  - _weirdness_ - отношение наблюдаемой вероятности к ожидаемой

## Хи-квадрат
Для каждого слова по двум корпусам можно построить таблицу сопряженности:

<table align="center">
    <tr>
        <td></td>
        <td>Target</td>
        <td>Reference</td>
        <td>Total (row sum)</td>
    </tr>
    <tr>
        <td>Word</td>
        <td>$O_{11}$=a</td>
        <td>$O_{12}$=b</td>
        <td>${T'}_1$=a+b</td>
    </tr>
    <tr>
        <td>Not word</td>
        <td>$O_{12}$=c-a</td>
        <td>$O_{22}$=d-b</td>
        <td>${T'}_2$=c+d-a-b</td>
    </tr>
    <tr>
        <td>Total (column sum)</td>
        <td>$T_1$=c</td>
        <td>$T_2$=d</td>
        <td>$N$=c+d</td>
    </tr>
</table>

Ожидаемую частоту оценим так:
$E_{ij}=\frac{T_i*{T'}_j}{N}$

Тогда для $\chi^2$ справедливо следующее:

$$\chi^2 = \sum{\frac{(O_{ij}-E_{ij})^2}{E_{ij}}} = \frac{N(O_{11}O_{22}-O_{12}O_{21})}{(O_{11}+O_{12})(O_{11}+O_{21})(O_{12}+O_{22})(O_{21}+O_{22})}$$


## Log-likelihood (ratio)

логарифмическая функция правдоподобия (Dunning 1993)

Наблюдаемые значения - $a$ и $b$, ожидаемые вычисляем так: $E_1 = \frac{c(a+b)}{c+d}$, $E_2 = \frac{d(a+b)}{c+d}$

"Значимость" слова:

$$G_2 = 2 \sum_i{O_i ln\frac{O_i}{E_i}}$$

## Weirdness

странность - интуитивная count-based метрика (Ahmad et al. 1999)

$$W = \frac{a / c}{(b + 1) / (d + V)}$$
где $V$ - размер словаря

## Выделение ключевых слов внутри текста
* RAKE
* TextRank
* C-value
* ...


# Графовые методы

## Матрица / граф совместной встречаемости

* задаем размер контекста: предложение, +-N слов и тд.
* считаем частоты пар слов


![Co-occurrence](kw-3.png)

(картинка из https://arxiv.org/abs/2106.06090)

## RAKE
- учитывает частоты отдельных слов и их совместную встречаемость
- делим текст на слова, затем по стоп-словам и маркерам границ фраз делим на фразы
- строим матрицу совместной встречаемости (все слова * все слова)


По матрице/графу совместной встречаемости вычисляем два значения:
- частота слова $f(w)$
- степень слова (сколько других слов встретилось с ним в контекстном окне) $d(w)$

$$score = \frac{d(w)}{f(w)}$$

## Использование RAKE

In [1]:
# установка
#!pip install python-rake

Итак, нам нужны стоп-слова (забудем пока о границах фраз) и токенизация -> NLTK.

In [2]:
# импорты
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [3]:
stop = stopwords.words('russian')

### Какой-нибудь текст

In [4]:
wiki_text = """
Мальтийский язык (собств. наименование — malti) — язык мальтийцев, официальный язык Республики Мальта (наряду с английским), один из официальных языков Европейского союза.
Относится к семитской семье афразийской (семито-хамитской) макросемьи языков. Язык близок к арабскому, особенно к его магрибскому диалекту, однако в настоящее время считается отдельным языком со своими синтаксическими, фонетическими и лексическими особенностями.

Лингвогеография
Число говорящих — 372 тыс. чел.

Статус мальтийского языка определяется в статье 5 главы I конституции Мальты:

Национальным языком Мальты является мальтийский.
Мальтийский и английский языки, а также и другой язык, который может быть утверждён Парламентом (по закону, одобренному не менее, чем 2/3 всех членов Палаты представителей), должны быть официальными языками Мальты, и Администрация может использовать во всех своих официальных целях любой из этих языков, при условии, что любое лицо вправе обратиться к Администрации на любом из официальных языков и ответ Администрации данному лицу должен быть дан на том же языке.
Языком судопроизводства должен быть мальтийский, при условии, что парламент может издать постановление об использовании английского языка в случаях, и на условиях, определяемых в данном постановлении.
Палата представителей может, регулируя свои собственные процессуальные нормы, определить язык или языки, которые должны использоваться в работе парламента и в протоколировании.
Большинство населения Мальты двуязычно. Мальтийский язык используется в повседневном общении, в парламенте, в церкви, в СМИ, в Интернете. В разговоре мальтийский часто смешивается с английским.

На мальтийский как на официальный язык ЕС должны переводиться все официальные документы ЕС. В Интернете существует мальтийская версия портала Европейского союза.

На мальтийском языке также говорят в Австралии, в среде выходцев из Мальты и их потомков. У них сложился свой диалект — Maltraljan (Maltese Australian).

История
Во время финикийской, позже карфагенской колонизации (800 до н. э. — 218 до н. э.) население мальтийских островов говорило на одном из финикийских (пунических) языков. На финикийском наречии продолжали говорить также в период римского (с 218 до н. э.), а после разделения империи на Западную и Восточную — «восточно-римского» (византийского) владычества (в 395—870 н. э.).

В 870 году острова были завоёваны арабами. Арабский язык распространился и среди коренного населения, в частности, благодаря своему родству с финикийским. При этом разговорная речь арабов и, соответственно, местного населения отличалась от классического письменного арабского. В этот период можно говорить о существовании двух диалектов: классический арабский письменный язык и сицилийско-арабский народно-разговорный язык, центром распространения которого стал Сицилийский эмират. После вторжения норманнов сицилийско-арабский язык был постепенно вытеснен с Сицилии, однако он сохранился на небольшом периферийном острове Мальта (хотя и испытал на себе сильное романское влияние). Именно от него и происходит нынешний мальтийский язык.

Новый этап начался с норманнским завоеванием Мальты в 1090 году и присоединением её к Сицилии. Язык администрации относился к романским, и в это время произошёл новый языковой семито-романский контакт. Полной романизации населения, как и в римскую эпоху, не произошло, так как арабы продолжительное время оставались на островах и оказывали значительное культурное и языковое влияние, однако уже тогда в язык попали первые романские заимствования. Арабы покинули Мальту через 100—150 лет, и вследствие этого произошло размежевание мальтийского и арабского.

Мальта была связана с Сицилией на протяжении 440 лет, переходила вместе с ней от одной феодальной юрисдикции к другой, и в течение этого времени количество романских заимствований увеличивалось. К XV веку относится первый известный литературный текст на мальтийском языке — поэма «Il Cantilena».

Наиболее сильное романское воздействие язык получил во время правления на Мальте рыцарского ордена Святого Иоанна — с 1530 по 1798 гг. В это время письменными языками были латынь и итальянский (сицилийский и классический), однако известны отдельные попытки писать и на мальтийском, например поэма Бонамико (Bonamico), 1672. Первая опубликованная книга на мальтийском языке — катехизис Виццино (Wizzino), датируется 1752 годом.

В 1800 году Мальта была присоединена к Великобритании. Несмотря на это, в течение XIX века английский язык не занимал той важной позиции на Мальте, которую занимает сегодня, официальным письменным языком оставался итальянский, разговорным — мальтийский.

Говоря об истории мальтийского языка, следует отметить деятельность Микиля Антона Вассалли (Mikiel Anton Vassalli, 1764—1829). Он составил словарь мальтийского языка и описание мальтийской грамматики, перевёл на мальтийский часть Нового Завета.

В 1921 году Мальте было предоставлено самоуправление, по конституции официальными языками стали итальянский и английский. В 1936 году мальтийский получил статус официального языка, взамен итальянского. Мальтийский и английский остались государственными языками и после провозглашения независимости Мальты в 1964 году.

1 мая 2004 года, в связи со вступлением Мальты в ЕС, мальтийский язык стал официальным языком ЕС.

Письменность
Мальтийский — единственный из семитских языков с письменностью на основе латинского алфавита.
В алфавит иногда включают букву c (сортируется после ċ), однако эта буква встречается только в заимствованных словах, в основном итальянского и английского происхождения, напр. camping — «кэмпинг» (с другой стороны, во многих словах c заменяется на k, напр. kompjuter — «компьютер», от англ. computer).

Лингвистическая характеристика
Просодия
Ударение в мальтийском, как правило, падает на предпоследний слог, если иной слог не является долгим.

Морфология
Грамматика мальтийского языка семитского типа, с романским влиянием. Прилагательные следуют за существительными, отсутствует наречие как самостоятельная форма, порядок слов достаточно гибок. Имеется определённый артикль. Как в арабском и иврите, артикль ставится и перед существительными, и перед прилагательными, напр. L-Art l-Imqaddsa — «Святая земля» (дословный перевод — «Земля Святая»; ср. также араб. الأرض المقدسة‎, ʾAl-ʾArḍ ʾal-Muqaddasa с тем же значением). Однако это правило относится только к семитским словам и не действует при употреблении существительных и прилагательных романского происхождения.

Существительные изменяются по числам. У существительных романского происхождения множественное число образуется добавлением -i или -jiet: lingwa («язык», ср. итал. lingua) — lingwi; arti («искусство», ср. итал. arte) — artijiet. С существительными семитского происхождения всё гораздо сложнее. Такие существительные делятся на правильные и неправильные. У правильных существительных множественное число образуется добавлением -iet / -jiet (соответствует -at в арабском и -ot в иврите) или -in (соответствует -im в иврите). У неправильных же существительных множественное число образуется изменением гласных внутри: ktieb («книга») — kotba, raġel («человек») — irġiel. Такая система очень развита в арабском и встречается в иврите.

Кроме множественного числа в мальтийском языке также имеется двойственное. Оно образуется с помощью суффикса -ejn или -ajn и употребляется при указании двух или парных предметов, напр. jum («день») — jumejn («два дня»), id («рука») — idejn («обе руки»).

В глаголах можно выделить семитский корень, состоящий из трёх согласных. Глагол спрягается добавлением к этому корню внутренних гласных, а также суффиксов и приставок. Пример: ktb — корень, означающий «писать», ktibna — «мы написали» (ср. араб. katabna, иврит. katavnu с теми же значениями). Существуют два времени: настоящее и прошедшее (перфект). Для выражения будущего используются формы настоящего времени с добавлением частиц или вспомогательных глаголов.

Особенностью мальтийской глагольной системы является то, что к романским по происхождению глаголам добавляются арабские суффиксы и приставки, напр. iddeċidejna — «мы решили», где (i)ddeċieda — глагол (ср. итал. decidere), -ejna — арабский элемент, образующий форму 1-го лица множественного числа прошедшего времени. В литературном арабском и иврите такое происходит крайне редко, однако подобные формы встречаются в некоторых арабских диалектах.

Лексика
Наряду с исконным слоем семитской лексики большую роль играют романские (в основном сицилийско-итальянские, реже тосканско-итальянские) заимствования.

В базовой лексике преобладают слова арабского происхождения, а для обозначения новых идей и предметов, в качестве терминов, относящихся к государственной деятельности, юриспруденции, образованию, искусству, литературе широко используются итальянизмы. Такие слова, как raġel («человек»), mara («женщина»), tifel («ребёнок»), dar («дом»), xemx («солнце»), sajf («лето») по происхождению арабские, а skola («школа»), gvern («правительство»), repubblika («республика»), re («король»), natura («природа»), pulizija («полиция»), ċentru («центр»), teatru («театр»), differenza («разница») — романские.

Итальянские по происхождению слова обычно отражают сицилийское произношение, в то время как литературный итальянский язык происходит из тосканского диалекта. Так, конечному -o в тосканском диалекте итальянского соответствует -u в мальтийском языке, как и в сицилийском диалекте, например, teatru, а не teatro. Также тосканскому -e соответствует -i, напр. arti — «искусство», fidi — «вера», lokali — «местный житель» (ср. литературные итальянские arte, fede, locale). Как и в некоторых романских языках, звук «ш» передается буквой x, в отличие от итальянского, где для этой цели используется диграф sc: ambaxxata — «посольство» (ср. итал. ambasciata), xena — «сцена» (ср. итал. scena).

Обычным делом в современном мальтийском языке является заимствование слов из английского, например, strajk — «забастовка» (от англ. strike), televixin — «телевидение» (от англ. television), tim — «команда» (от англ. team). Некоторые слова сохраняют английское написание, например, union — «союз», «объединение», leave — «отпуск», bonus — «премия»."""

## Построение модели

In [5]:
# Инициализируем анализатор списком стоп-слов
rake = RAKE.Rake(stop)

In [6]:
# Запускаем
wiki_kw_list = rake.run(wiki_text, maxWords=3, minFrequency=2)

In [7]:
wiki_kw_list

[('мальтийский', 1.5714285714285714),
 ('например', 1.5),
 ('романским', 1.3333333333333333),
 ('мальтийском', 1.3333333333333333),
 ('арабском', 1.25),
 ('английским', 1.0),
 ('однако', 1.0),
 ('условии', 1.0),
 ('н', 1.0),
 ('э', 1.0),
 ('сицилии', 1.0),
 ('напр', 1.0),
 ('англ', 1.0),
 ('существительными', 1.0),
 ('иврите', 1.0),
 ('ср', 1.0),
 ('итал', 1.0),
 ('raġel', 1.0),
 ('«человек»', 1.0),
 ('teatru', 1.0)]

Похоже, для отдельных слов можно применить лемматизацию и станет лучше.
А ещё токенизировать красиво.

In [8]:
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize

In [9]:
m = MorphAnalyzer()
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [10]:
rake.run(normalize_text(wiki_text), maxWords=3, minFrequency=2)

[('мальтийский язык', 3.9583333333333335),
 ('язык', 2.2083333333333335),
 ('мальта', 2.1),
 ('арабский', 2.090909090909091),
 ('романский', 2.0),
 ('итальянский', 2.0),
 ('мальтийский', 1.75),
 ('относиться', 1.6666666666666667),
 ('существительное', 1.6),
 ('английский', 1.5714285714285714),
 ('время', 1.5),
 ('финикийский', 1.3333333333333333),
 ('например', 1.2),
 ('однако', 1.0),
 ('условие', 1.0),
 ('н', 1.0),
 ('э', 1.0),
 ('говорить', 1.0),
 ('сицилия', 1.0),
 ('англ', 1.0),
 ('иврит', 1.0),
 ('сравнить', 1.0),
 ('значение', 1.0),
 ('итал', 1.0),
 ('неправильный', 1.0),
 ('raġel', 1.0),
 ('« человек »', 1.0),
 ('приставка', 1.0),
 ('teatru', 1.0)]

Попробуем какой-нибудь другой жанр, например, **новости**.

In [11]:
bbc_text = """
Королева Британии Елизавета II приняла решение отказаться от одежды и аксессуаров из натурального меха, сообщил Букингемский дворец. Некоторые королевские наряды уже перешиты.
О том, что королева отказывается от натурального меха, написала в своей книге "Другая сторона монеты: королева, модельер и платяной шкаф" личный костюмер королевы Анджела Келли.
"Если Ее Величеству придется посещать какое-либо мероприятие в очень холодную погоду, с 2019 года, чтобы не замерзнуть, она будет надевать одежду из искусственного меха", - рассказала личный костюмер королевы Анджела Келли.
Пресс-секретарь Букингемского дворца подтвердил, что для пошива новой одежды для королевы при необходимости будет использоваться только искусственный мех.
При этом, как уточнили в Букингемском дворце, королева продолжит носить те меховые изделия, которые были приобретены или созданы для нее ранее - от шляп до церемониальных мантий.
В своей книге Анджела Келли рассказывает, что мех норки на пальто, которое было на королеве во время ее визита в Словакию в 2008 году, сейчас заменен на искусственный.
Королеву неоднократно критиковали за то, что она продолжала носить меховые изделия, когда от использования меха отказались почти все модные дома, признав использование меха для изготовления одежды и аксессуаров жестоким решением.
"""

In [12]:
rake.run(bbc_text, minFrequency=2, maxWords=3)

[('натурального меха', 4.4)]

In [13]:
rake.run(normalize_text(bbc_text), minFrequency=2, maxWords=3)

[('натуральный мех', 4.285714285714286),
 ('искусственный мех', 3.9523809523809526),
 ('королева', 1.8333333333333333),
 ('который', 1.0)]

Ну и наконец сложно формализуемый текст.

In [14]:
fiction_text = """
Ваши превосходительства, высокородия, благородия, граждане!
– –
Что есть Русская Империя наша?
Русская Империя наша есть географическое единство, что значит: часть известной планеты. И Русская Империя заключает: во-первых – великую, малую, белую и червонную Русь; во-вторых – грузинское, польское, казанское и астраханское царство; в-третьих, она заключает… Но – прочая, прочая, прочая.
Русская Империя наша состоит из множества городов: столичных, губернских, уездных, заштатных; и далее: – из первопрестольного града и матери градов русских.
Град первопрестольный – Москва; и мать градов русских есть Киев.
Петербург, или Санкт-Петербург, или Питер (что – то же) подлинно принадлежит Российской Империи. А Царьград, Константиноград (или, как говорят, Константинополь), принадлежит по праву наследия. И о нем распространяться не будем.
Распространимся более о Петербурге: есть – Петербург, или Санкт-Петербург, или Питер (что – то же). На основании тех же суждений Невский Проспект есть петербургский Проспект.
Невский Проспект обладает разительным свойством: он состоит из пространства для циркуляции публики; нумерованные дома ограничивают его; нумерация идет в порядке домов – и поиски нужного дома весьма облегчаются. Невский Проспект, как и всякий проспект, есть публичный проспект; то есть: проспект для циркуляции публики (не воздуха, например); образующие его боковые границы дома суть – гм… да:… для публики. Невский Проспект по вечерам освещается электричеством. Днем же Невский Проспект не требует освещения.
Невский Проспект прямолинеен (говоря между нами), потому что он – европейский проспект; всякий же европейский проспект есть не просто проспект, а (как я уже сказал) проспект европейский, потому что… да…
Потому что Невский Проспект – прямолинейный проспект.
Невский Проспект – немаловажный проспект в сем не русском – столичном – граде. Прочие русские города представляют собой деревянную кучу домишек.
И разительно от них всех отличается Петербург.
Если же вы продолжаете утверждать нелепейшую легенду – существование полуторамиллионного московского населения – то придется сознаться, что столицей будет Москва, ибо только в столицах бывает полуторамиллионное население; а в городах же губернских никакого полуторамиллионного населения нет, не бывало, не будет. И согласно нелепой легенде окажется, что столица не Петербург.
Если же Петербург не столица, то – нет Петербурга. Это только кажется, что он существует.
Как бы то ни было, Петербург не только нам кажется, но и оказывается – на картах: в виде двух друг в друге сидящих кружков с черной точкою в центре; и из этой вот математической точки, не имеющей измерения, заявляет он энергично о том, что он – есть: оттуда, из этой вот точки, несется потоком рой отпечатанной книги; несется из этой невидимой точки стремительно циркуляр.
"""

In [15]:
rake.run(normalize_text(fiction_text), minFrequency=2, maxWords=3)

[('русский империя наш', 8.666666666666666),
 ('мать град русский', 7.866666666666667),
 ('невский проспект', 4.34453781512605),
 ('европейский проспект', 4.0588235294117645),
 ('циркуляция публика', 3.666666666666667),
 ('санкт-петербург', 3.4),
 ('петербург', 1.4),
 ('прочий', 1.0),
 ('столичный', 1.0),
 ('москва', 1.0),
 ('питер', 1.0),
 ('говорить', 1.0),
 ('столица', 1.0),
 ('казаться', 1.0),
 ('…', 0)]

## TextRank
— алгоритм, основанный на PageRank, но для текстов и слов, а не документов и запросов:
- строим граф совместной встречаемости
- можно ввести веса на рёбрах  $w_{ij}$ (напр, нормализованные частоты и т.п.)
- запускаем случайное блуждание: итеративно обновляем веса вершин V

Обозначим $In(V_i)$ все вершины, **из** которых есть ребро в $V_i$, а $Out(V_i)$ - все вершины, **в** которые есть ребро из $V_i$

$$WS(V_i) = (1-d) + d* \sum_{V_j\in{In(V_i)}}{\frac{w_{ji}}{\sum_{V_k\in Out(V_j)}{w_{ik}}}WS(V_j)}$$

$d$ – damping factor / коэффициент затухания, может быть установлен между нулем и единицей, моделирует вероятность случайного перехода от одной вершины графа к другой; означает вероятность того, что пользователь, зашедший на страницу, перейдет по одной из ссылок, содержащейся на этой странице, а не прекратит путешествие по сети, в классической формуле обычно он равен 0,85.

первоначально (при нулевой итерации) веса расставляются случайным образом (например, $1/N$)

На следующих итерациях используется вес всех входящих ссылок, который представляет собой вес с предыдущей итерации делённый на количество исходящих ребер $Out(V_j)$.

Одна из библиотек, где реализован TextRank, — **summa**. Она даже позволяет посмотреть на граф, по которому мы блуждаем.

In [19]:
!pip install summa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 276.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54389 sha256=e599b79708808bbd6eefd15f6b4383a502daaf1d3c3d823657c07fb03179009c
  Stored in directory: /Users/rhubarb/Library/Caches/pip/wheels/ed/2c/5f/a0ccc5955d44d2cea78729f4425e73f818d2629517f7af0f8b
Successfully built summa


In [20]:
from summa import keywords

In [21]:
keywords.keywords(normalize_text(wiki_text), language='russian', additional_stopwords=stop, scores=True)

[('мальтийский язык', 0.34096872791906857),
 ('например', 0.20246045005834304),
 ('арабский', 0.1444018809427446),
 ('итальянский', 0.1338216554541493),
 ('слово', 0.12718736962317045),
 ('сравнить', 0.11563441850916602),
 ('республика мальта', 0.1101687684337988),
 ('свой', 0.10566458851440774),
 ('время', 0.10521307029344402),
 ('семитский', 0.1030435699932755),
 ('диалект', 0.09983966498642755),
 ('официальный', 0.09818543703684926),
 ('романский влияние', 0.09787401386018939),
 ('араб', 0.09676847027568375),
 ('существительное', 0.09364037135534613),
 ('использовать', 0.09286079119860394),
 ('использоваться', 0.09286079119860394),
 ('однако', 0.09238708169998006),
 ('население', 0.09214508683353118),
 ('новый', 0.09147439875706287),
 ('соответствовать', 0.08992145502255411),
 ('происхождение', 0.08908545847043013),
 ('сицилийско', 0.08789407062030641),
 ('коренной', 0.0848968963057687),
 ('корень', 0.0848968963057687),
 ('форма', 0.0840134963305284),
 ('глагол', 0.08128294993943025

### Соседи в графе

In [22]:
G = keywords.get_graph(normalize_text(wiki_text))
G.node_neighbors['язык']
# viz - networkx

['мальтийский',
 'собство',
 'malti',
 'мальтиец',
 'официальный',
 'республика',
 'европейский',
 'макросемья',
 'язык',
 'близкий',
 'отдельный',
 'с',
 'определяться',
 'национальный',
 'мальта',
 'английский',
 'а',
 'другой',
 'который',
 'этот',
 'при',
 'и',
 'же',
 'судопроизводство',
 'в',
 'определить',
 'или',
 'использоваться',
 'ес',
 'также',
 'пунический',
 'на',
 'арабский',
 'распространиться',
 'письменный',
 'разговорный',
 'центр',
 'быть',
 'новый',
 'сицилия',
 'администрация',
 'попасть',
 'поэма',
 'воздействие',
 'получить',
 'катехизис',
 'не',
 'оставаться',
 'следовать',
 'стать',
 'взамен',
 'государственный',
 'семитский',
 'lingwa',
 'сравнить',
 'итальянский',
 'происходить',
 'как',
 'романский',
 'звук',
 'являться']

# Другие методы

## Embeddings-based

- выделяем фразы-кандидаты
- векторизуем их
- векторизуем текст
- ранжируем кандидатовв по близости к тексту

## Language model-based
* идея контрастивных методов:
  * сравниваем вероятности слов фразы и всей фразы в документе
  * сравниваем вероятности фразы в документе и целом корпусе

# Supervised методы

## Представление данных
- фразы + классы (да/нет)
- слова в тексте + тег для каждого слова

## ML-based подходы

- бинарная классификация

- sequence labelling / разметка последовательности

## Пример

`EM-алгоритм (англ. Expectation-maximization (EM) algorithm) — алгоритм, используемый в математической статистике для нахождения оценок максимального правдоподобия параметров вероятностных моделей, в случае, когда модель зависит от некоторых скрытых переменных.`

|кандидат|класс|
|-|-|
|`EM-алгоритм` | 1 |
|`алгоритм` | 0 |


Соответствие токенам:

`1 0 0 ...`

## KeyBERT

- использует эмбеддинг слова/фразы и текста
- не только BERT-style модели, но и более ранние эмбеддинги: word2vec, flair ...
- слово авторам: "KeyBERT is by no means unique"

## KeyBERT in action

In [23]:
!pip3 install keybert

In [24]:
from keybert import KeyBERT

/Users/rhubarb/Hse/hse-nlp/hse_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
# загрузим мультиязычную модельку для sentence-transformers
kw_model = KeyBERT('clips/mfaq')

/Users/rhubarb/Hse/hse-nlp/hse_env/lib/python3.9/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [26]:
kw_model.extract_keywords(wiki_text)

[('мальтийский', 0.9655),
 ('мальтийского', 0.9651),
 ('сицилийский', 0.9651),
 ('латынь', 0.9651),
 ('мальтийском', 0.965)]

In [27]:
# Словосочетания / n-граммы + лемматизация
kw_model.extract_keywords(normalize_text(wiki_text), keyphrase_ngram_range=(1, 2), stop_words=None)

[('грамматика мальтийский', 0.9768),
 ('мальтийский грамматика', 0.9756),
 ('словарь мальтийский', 0.9742),
 ('мальтийский глагольный', 0.9741),
 ('язык мальта', 0.9741)]

In [28]:
# подсветка
kw_model.extract_keywords(wiki_text, keyphrase_ngram_range=(1, 2), stop_words=None, highlight=True)

Мальтийский язык собств наименование malti язык официальный язык Республики Мальта наряду английским один из 
официальных языков Европейского союза Относится семитской семье афразийской семито хамитской макросемьи языков Язык
близок арабскому особенно его магрибскому диалекту однако настоящее время считается отдельным языком со своими 
синтаксическими фонетическими лексическими особенностями Лингвогеография Число говорящих 372 тыс чел Статус 
мальтийского языка определяется статье главы конституции Мальты Национальным языком Мальты мальтийский Мальтийский 
английский языки также другой язык который может быть утверждён Парламентом по закону одобренному не менее чем всех
членов Палаты представителей должны быть официальными языками Мальты Администрация может использовать во всех своих
официальных целях любой из этих языков при условии что любое лицо вправе обратиться Администрации на любом из 
официальных языков ответ Администрации данному лицу должен быть дан на том же языке Языком судопроизводства должен 
быть мальтийский при условии что парламент может издать постановление об использовании английского языка случаях на
условиях определяемых данном постановлении Палата представителей может регулируя свои собственные процессуальные 
нормы определить язык или языки которые должны использоваться работе парламента протоколировании Большинство 
населения Мальты двуязычно Мальтийский язык используется повседневном общении парламенте церкви СМИ Интернете 
разговоре мальтийский часто смешивается английским На мальтийский как на официальный язык ЕС должны переводиться 
все официальные документы ЕС Интернете существует мальтийская версия портала Европейского союза На мальтийском 
языке также говорят Австралии среде выходцев из Мальты их потомков них сложился свой диалект Maltraljan Maltese 
Australian История Во время финикийской позже карфагенской колонизации 800 до 218 до население мальтийских островов
говорило на одном из финикийских пунических языков На финикийском наречии продолжали говорить также период римского
218 до после разделения империи на Западную Восточную восточно римского византийского владычества 395 870 870 году 
острова были завоёваны арабами Арабский язык распространился среди коренного населения частности благодаря своему 
родству финикийским При этом разговорная речь арабов соответственно местного населения отличалась от классического 
письменного арабского этот период можно говорить существовании двух диалектов классический арабский письменный язык
сицилийско арабский народно разговорный язык центром распространения которого стал Сицилийский эмират После 
вторжения норманнов сицилийско арабский язык был постепенно вытеснен Сицилии однако он сохранился на небольшом 
периферийном острове Мальта хотя испытал на себе сильное романское влияние Именно от него происходит нынешний 
мальтийский язык Новый этап начался норманнским завоеванием Мальты 1090 году присоединением её Сицилии Язык 
администрации относился романским это время произошёл новый языковой семито романский контакт Полной романизации 
населения как римскую эпоху не произошло так как арабы продолжительное время оставались на островах оказывали 
значительное культурное языковое влияние однако уже тогда язык попали первые романские заимствования Арабы покинули
Мальту через 100 150 лет вследствие этого произошло размежевание мальтийского арабского Мальта была связана 
Сицилией на протяжении 440 лет переходила вместе ней от одной феодальной юрисдикции другой течение этого времени 
количество романских заимствований увеличивалось XV веку относится первый известный литературный текст на 
мальтийском языке поэма Il Cantilena Наиболее сильное романское воздействие язык получил во время правления на 
Мальте рыцарского ордена Святого Иоанна 1530 по 1798 гг это время письменными языками были латынь итальянский 
сицилийский классический однако известны отдельные попытки писать на мальтийском например поэма Бонамико Bonamico 
1672 Первая опубликованная книга на мальтий

[('грамматика мальтийского', 0.9737),
 ('язык мальтийцев', 0.9727),
 ('malti язык', 0.972),
 ('словарь мальтийского', 0.971),
 ('языком мальты', 0.9707)]

Подробнее: https://maartengr.github.io/KeyBERT/guides/embeddings.html#hugging-face-transformers 